In [1]:
import csv 
import requests
import os
import pandas as pd
from openai import OpenAI
from bs4 import BeautifulSoup
from typing import Dict, List
from urllib.parse import urlparse
import tkinter as tk
from tkinter import filedialog

In [2]:
client = OpenAI(
    api_key = 'sk-svcacct-XNnYiDVzrJ3Tb8q5SSE_wy2n6L6AAks3onXlCHR2iifEOomzqilw3-cXHG6ZD6aD4rqcBL3R8T3BlbkFJUwgZhxvBp9XVBnCFYQvUFQtNzG--VDzLNKkudsq4YP4cNveJm1r9stlPt57-drV-cKUPGl5-wA',
)

In [19]:
csv_url = input("Enter the URL of the CSV file: ")
df = pd.read_csv(csv_url)
df.head()

,First Name,Last Name,Title,Company,Company Name for Emails,Email,Email Status,Email Confidence,Primary Email Catch-all Status,Primary Email Last Verified At,...,ZB Free Email,ZB MX Found,ZB MX Record,ZB SMTP Provider,ZB Did You Mean,ZB City,ZB Region/State,ZB Zip Code,ZB Country,ZB Last Known Activity
0,Francois-Xavier,Lemire,Co-Founder & Chief Technology Officer,Mercantile,Mercantile,fx@getmercantile.com,Likely to engage,NaN,Catch-all,2024-09-24T05:45:40+00:00,...,False,True,aspmx.l.google.com,g-suite,NaN,NaN,NaN,NaN,NaN,NaN
1,Casey,Witkowicz,President & CEO,RYCOM,RYCOM,casey.witkowicz@rycom.com,Likely to engage,NaN,Not Catch-all,2024-10-04T01:21:59+00:00,...,False,True,rycom-com.mail.protection.outlook.com,microsoft,NaN,NaN,NaN,NaN,NaN,30
2,James,Rockwood,Founder & CEO,CapIntel,CapIntel,rockwood@capintel.com,Likely to engage,NaN,Not Catch-all,2024-10-04T01:24:26+00:00,...,False,True,capintel-com.mail.protection.outlook.com,microsoft,NaN,NaN,NaN,NaN,NaN,NaN
3,Michael,Ginsberg,CEO,Echoworx,Echoworx,ginsberg@echoworx.com,Likely to engage,NaN,Not Catch-all,2024-10-01T03:25:39+00:00,...,False,True,echoworx-com.mail.protection.outlook.com,microsoft,NaN,NaN,NaN,NaN,NaN,60
4,Sasha,Cucuz,CEO,Greybrook Realty Partners,Greybrook Realty,sasha.cucuz@greybrook.com,Likely to engage,NaN,Not Catch-all,2024-10-02T15:38:53+00:00,...,False,True,d140661a.ess.barracudanetworks.com,barracuda,NaN,NaN,NaN,NaN,NaN,180


In [20]:
def scrape_websites(Website):
    
    try:
        response = requests.get(Website, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        data = {
            'url': Website,
            'title': soup.title.string if soup.title else 'No title found',
            'meta_description': soup.find('meta', attrs={'name': 'description'})['content'] if soup.find('meta', attrs={'name': 'description'}) else 'No meta description found',
            'headings': [h.text for h in soup.find_all(['h1', 'h2', 'h3'])[:5]],
            'paragraphs': [p.text for p in soup.find_all('p')[:20]]
        }

        return data
    
    except Exception as e:
        return "error"

In [21]:
df['website_data'] = df['Website'].apply(scrape_websites)
df.head()

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


,First Name,Last Name,Title,Company,Company Name for Emails,Email,Email Status,Email Confidence,Primary Email Catch-all Status,Primary Email Last Verified At,...,ZB MX Found,ZB MX Record,ZB SMTP Provider,ZB Did You Mean,ZB City,ZB Region/State,ZB Zip Code,ZB Country,ZB Last Known Activity,website_data
0,Francois-Xavier,Lemire,Co-Founder & Chief Technology Officer,Mercantile,Mercantile,fx@getmercantile.com,Likely to engage,NaN,Catch-all,2024-09-24T05:45:40+00:00,...,True,aspmx.l.google.com,g-suite,NaN,NaN,NaN,NaN,NaN,NaN,"{'url': 'http://www.getmercantile.com', 'title..."
1,Casey,Witkowicz,President & CEO,RYCOM,RYCOM,casey.witkowicz@rycom.com,Likely to engage,NaN,Not Catch-all,2024-10-04T01:21:59+00:00,...,True,rycom-com.mail.protection.outlook.com,microsoft,NaN,NaN,NaN,NaN,NaN,30,"{'url': 'http://www.rycom.com', 'title': 'RYCO..."
2,James,Rockwood,Founder & CEO,CapIntel,CapIntel,rockwood@capintel.com,Likely to engage,NaN,Not Catch-all,2024-10-04T01:24:26+00:00,...,True,capintel-com.mail.protection.outlook.com,microsoft,NaN,NaN,NaN,NaN,NaN,NaN,"{'url': 'http://www.capintel.com', 'title': 'M..."
3,Michael,Ginsberg,CEO,Echoworx,Echoworx,ginsberg@echoworx.com,Likely to engage,NaN,Not Catch-all,2024-10-01T03:25:39+00:00,...,True,echoworx-com.mail.protection.outlook.com,microsoft,NaN,NaN,NaN,NaN,NaN,60,"{'url': 'http://www.echoworx.com', 'title': 'E..."
4,Sasha,Cucuz,CEO,Greybrook Realty Partners,Greybrook Realty,sasha.cucuz@greybrook.com,Likely to engage,NaN,Not Catch-all,2024-10-02T15:38:53+00:00,...,True,d140661a.ess.barracudanetworks.com,barracuda,NaN,NaN,NaN,NaN,NaN,180,"{'url': 'http://www.greybrook.com', 'title': '..."


In [29]:
def gpt_prompt_generate_perssonalized_from_website(row):
    
    website_data = row['website_data']
    company_name = row['Company']
    Keywords_data = row['Keywords']
    Industry_data = row['Industry']
        
    prompt = f""" Generate a compelling, casual, and friendly introductory line for an email. 
    The line should start with something similar to "I was on your website and saw {company_name}" and highlight their services, values, and how their success and growth, based on data from {website_data}, and that they are contributing to rapid expansion in their industry based on {Keywords_data} or {Industry_data}. 
    The tone should be conversational, recognizing their achievements and positioning the email as relevant to their success. Keep the sentence under 30 words.
    
    Example Outputs:
    I was on your website and saw Acme Corp is making waves in the financial services industry, especially with your focus on digital payment solutions, which seems to be driving impressive growth.
    I was on your website and saw BlueTech Solutions is really thriving in healthcare technology, thanks to your focus on streamlining patient data management. It’s clear your growth is on the rise!
    I checked out your website and noticed Growth Dynamics is growing fast in logistics management, with your dedication to process automation clearly fueling that expansion.
    I saw on your website that InnovateX is growing rapidly in e-commerce software—your commitment to improving customer experience through AI seems to be paying off in a big way.
    I was on your website and saw how HealthStream is quickly expanding in patient care analytics, especially with your focus on data-driven decision-making for healthcare providers—it’s impressive to see how fast you’re growing!
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that writes personalized outbound sales emails for Litespace who is a startup digitizing human resources and driving performance through improving employee experience."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=200,
            n=1,
            stop=None,
            temperature=1.0
            )

        generated_email = response.choices[0].message.content
        return generated_email

    except Exception as e:
         return f"Error: {e}"

In [30]:
df['Message'] = df.apply(gpt_prompt_generate_perssonalized_from_website, axis=1)
df.head()

,First Name,Last Name,Title,Company,Company Name for Emails,Email,Email Status,Email Confidence,Primary Email Catch-all Status,Primary Email Last Verified At,...,ZB MX Record,ZB SMTP Provider,ZB Did You Mean,ZB City,ZB Region/State,ZB Zip Code,ZB Country,ZB Last Known Activity,website_data,Message
0,Francois-Xavier,Lemire,Co-Founder & Chief Technology Officer,Mercantile,Mercantile,fx@getmercantile.com,Likely to engage,NaN,Catch-all,2024-09-24T05:45:40+00:00,...,aspmx.l.google.com,g-suite,NaN,NaN,NaN,NaN,NaN,NaN,"{'url': 'http://www.getmercantile.com', 'title...",I was on your website and saw Mercantile is re...
1,Casey,Witkowicz,President & CEO,RYCOM,RYCOM,casey.witkowicz@rycom.com,Likely to engage,NaN,Not Catch-all,2024-10-04T01:21:59+00:00,...,rycom-com.mail.protection.outlook.com,microsoft,NaN,NaN,NaN,NaN,NaN,30,"{'url': 'http://www.rycom.com', 'title': 'RYCO...",I was on your website and saw RYCOM driving ra...
2,James,Rockwood,Founder & CEO,CapIntel,CapIntel,rockwood@capintel.com,Likely to engage,NaN,Not Catch-all,2024-10-04T01:24:26+00:00,...,capintel-com.mail.protection.outlook.com,microsoft,NaN,NaN,NaN,NaN,NaN,NaN,"{'url': 'http://www.capintel.com', 'title': 'M...",I was on your website and saw CapIntel's subst...
3,Michael,Ginsberg,CEO,Echoworx,Echoworx,ginsberg@echoworx.com,Likely to engage,NaN,Not Catch-all,2024-10-01T03:25:39+00:00,...,echoworx-com.mail.protection.outlook.com,microsoft,NaN,NaN,NaN,NaN,NaN,60,"{'url': 'http://www.echoworx.com', 'title': 'E...",I was on your website and saw Echoworx is lead...
4,Sasha,Cucuz,CEO,Greybrook Realty Partners,Greybrook Realty,sasha.cucuz@greybrook.com,Likely to engage,NaN,Not Catch-all,2024-10-02T15:38:53+00:00,...,d140661a.ess.barracudanetworks.com,barracuda,NaN,NaN,NaN,NaN,NaN,180,"{'url': 'http://www.greybrook.com', 'title': '...",I was on your website and saw Greybrook Realty...


In [31]:
root = tk.Tk()
root.withdraw()
save_folder = filedialog.askdirectory(title="Select a folder to save the file")

if save_folder:
    file_path = os.path.join(save_folder, "result.csv")
    df.to_csv(file_path, encoding='utf-8-sig', index=False)
    print(f"File saved to: {file_path}")
else:
    print("No folder selected. File not saved.")

File saved to: C:/Users/Karis/Litespace/Email Outreach/CEO Outreach (October 17th 2024)/Custom subject line +message\result.csv
